# Instagram Clone User Data Analysis

\*\*Use the ig\_clone database\*\*

In [3]:
USE ig_clone
GO

Commands completed successfully.

Total execution time: 00:00:00.014

## **Rewarding the Oldest Customers**

Let's reward the Instagram users who have been around the longest. For that, we wish to find the 10 oldest customers and reward them.

In [4]:
SELECT TOP(10) * FROM users 
ORDER BY created_at ASC;

(10 rows affected)

Total execution time: 00:00:00.129

id,username,created_at
80,Darby_Herzog,2016-05-06 00:14:21.190
67,Emilio_Bernier52,2016-05-06 13:04:29.960
63,Elenor88,2016-05-08 01:30:40.677
95,Nicole71,2016-05-09 17:30:22.370
38,Jordyn.Jacobson2,2016-05-14 07:56:25.837
71,Nia_Haag,2016-05-14 15:38:50.230
40,Rafael.Hickle2,2016-05-19 09:51:25.780
58,Aurelie71,2016-05-31 06:20:56.910
88,Clint27,2016-06-02 21:40:09.557
91,Bethany20,2016-06-03 23:31:53.323
